In [1]:
Dataset_url = "https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia"

In [2]:
# Upload Kaggle json file
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [00:12<00:00, 208MB/s]
100% 2.29G/2.29G [00:12<00:00, 194MB/s]


In [4]:
!unzip -q chest-xray-pneumonia.zip

In [5]:
!pip install -q timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.4 MB/s eta 0:00:00


In [10]:
import timm
from fastai.vision.all import *
import os
from fastai.data.external import *

In [7]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [8]:
# Define paths to your train, validation, and test data
train_path = Path('/content/chest_xray/train')
val_path = Path('/content/chest_xray/val')
test_path = Path('/content/chest_xray/test')

In [12]:
!ls

chest_xray  chest-xray-pneumonia.zip  kaggle.json  sample_data


In [14]:
!mv chest_xray/val/NORMAL/* chest_xray/train/NORMAL/

In [15]:
!mv chest_xray/val/PNEUMONIA/* chest_xray/train/PNEUMONIA/

In [16]:
data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

In [17]:
data = data.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())

In [18]:
# Get DataLoaders from the datablock
dls = data.dataloaders(train_path)

In [19]:
# Load ResNet18 model without pre-trained weights using Timm
model = timm.create_model('resnet18', pretrained=False, num_classes=dls.c)

In [20]:
# Create a Fastai Learner
learn = Learner(dls, model, metrics=accuracy)

In [21]:
dls.c

2

In [22]:
# Train the model
learn.fine_tune(epochs=5)

epoch,train_loss,valid_loss,accuracy,time
0,0.402367,6.386751,0.248566,01:26


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


epoch,train_loss,valid_loss,accuracy,time
0,0.299222,3.621189,0.309751,01:20
1,0.281085,0.677297,0.577438,01:21
2,0.274890,1.736789,0.323136,01:19
3,0.256325,0.144520,0.953155,01:22
4,0.240165,0.216335,0.910134,01:21


In [23]:
# Evaluate on the test set
test_dl = dls.test_dl(get_image_files(test_path))
learn.validate(dl=test_dl)

(#2) [None,None]

In [24]:
# Load the test data
test_files = get_image_files(test_path)

In [25]:
# Create a DataLoader for the test set
test_dl = dls.test_dl(test_files, with_labels=True)  # Set with_labels=False if test set has no labels

In [26]:
# Validate the model on the test set
results = learn.validate(dl=test_dl)

In [27]:
# Print results
print(results)

[0.2926744520664215, 0.8846153616905212]


In [28]:
# Print formatted results
print(f"Loss: {results[0]:.4f}, Accuracy: {results[1] * 100:.2f}%")

Loss: 0.2927, Accuracy: 88.46%
